In [1]:
# this block initializes the structs for storing acitivities and patients

import os
from datetime import datetime
import pandas as pd
import time
from patient import Patient, Activity


In [2]:
src = './data/HOA Data'
# to-do

excel = './data/HOA Data/chi_unix.xlsx'
excel = pd.read_excel(excel)


activities = []
ids = []
patients = []

for i in range(len(excel)):
    row = excel.iloc[i]
    if row['Subject Index'] not in ids:
        ids.append(row['Subject Index'])

for id in ids:
    # fetch all activities for each patients
    activities = []
    min_time = -1
    for i in range(len(excel)):
        act = excel.iloc[i]
        if act['Subject Index'] == id:
            if min_time <1 or min_time>=act['Record Time']:
                min_time = act['Record Time']
            activities.append(Activity(
                act['Activity'], act['Length'], act['Record Time'], act['Subject Index']))
            
    patient = Patient(id, 0, activities, [None], 0, min_time)
    patients.append(patient)



In [3]:
#load in csvs
src = './data/HOA Data'
csv_file = []


for path, subdirs, files in os.walk(src):
    for name in files:  # for every file in this directory
        file = os.path.join(path, name)

        if 'ACC.csv' in file:  # if file name contains acc.csv, read data
            acc = pd.read_csv(file)
            csv_file.append(acc)  # append to a list


In [9]:
for patient in patients:
    #for every unique patient - cross reference csvs
    buffer = 2*60*60
    for csv in csv_file:
        time = float(csv.columns.values[0])
        if time >= (patient.excel_time - buffer) or time <= (patient.excel_time +buffer):  # within buffer
            
            patient.clean_acc(csv)
            patient.csv_time = time
        csv_file-=csv
    print(patient.tldr())



ValueError: Unable to coerce to Series, length must be 3: given 39